Notebook was executed on kaggle server using GPU. Before executing the notebook have to add data on the notebook 
https://www.kaggle.com/zuozhaorui/celeba.

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
import os
import random
import shutil
import os
import cv2
import math

from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFilter
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [12]:
# To run the model on the GPU
torch.cuda.get_device_name(0)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
#### Function sample_data creates a  subset of main dataset for experimentation. In this case it is 100
def sample_data(source_folder_path, parent_dir):
    files_list = []
    for root, dirs, files in os.walk(source_folder_path):
        for file in files: 
            files_list.append(os.path.join(root, file))
            
    selected_files = random.sample(files_list, 100)
    directory = "dataset100"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    dest_path1 = path

    for src_path in selected_files:
        shutil.copy(src_path, os.path.join(dest_path1, os.path.basename(src_path)))
    return dest_path1

def create_directory(parent_dir, name):
    directory = name
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    return path

#### Function move_multiple transfers a specific number of images from a source folder to destination folder.
def move_multiple(source_path, detination_path,number):
    files_list = []
    for root, dirs, files in os.walk(source_path):
        for file in files: 
            files_list.append(file)
        
    selected_files = random.sample(files_list, number)
    source_folder = source_path + "/"
    destination_folder = detination_path + "/"
    #print(selected_files)

    for file in selected_files:
        source = source_folder + file
        destination = destination_folder + file
        shutil.move(source,destination)
    return source_folder

#### Function move_all transfers all the images from source folder to destination folder
def move_all(source_path, detination_path):
    source_folder = source_path + "/"
    destination_folder = detination_path + "/"
    for file in os.listdir(source_folder):
        source = source_folder + file
        destination = destination_folder + file
        if os.path.isfile(source):
            shutil.move(source,destination)

In [14]:
# get_cordinates helper function will take Length and Breadth of the image & generate one of the corner cordinates of patch image
# get_lengths_patch will return the patch length given the dimension of the image and cordinate of the corner of the patch

def get_cordinates(L, B, alpha):
    x = random.randint(0, math.floor(alpha * L))
    y = random.randint(0, math.floor(alpha * B))
    return x,y

def get_lengths_patch(L, x, beta):
    l = random.randint(math.floor(beta * L), L)
    while(x + l > L):
        l = random.randint(math.floor(beta * L), L)
    return l
# prepare_data_set method creates images by pasting the white patches of randomly generated sizes on the input images at random positions

def prepare_data_set(input_folder_path, output_folder_path, num_copies):
    #data_descripition_file = open(os.path.join(output_folder_path, 'data_description.csv'), "w")
    c=0
    for img_name in os.listdir(input_folder_path):
        c+=1
        if c%1000==0:
            print(c)
        for i in range(num_copies):
            image = Image.open(os.path.join(input_folder_path, img_name))
            L,B = image.size
            x,y = get_cordinates(L, B, 0.9)
            l = get_lengths_patch(L, x, 0.1 )
            b = get_lengths_patch(B, y, 0.1)
            patch_rect = np.ones((l,b,3),np.uint8)
            cv2.imwrite(os.path.join(output_folder_path, 'patch_image.png'), patch_rect)
            patch_rect = Image.open(os.path.join(output_folder_path, 'patch_image.png'))
            os.remove(os.path.join(output_folder_path, 'patch_image.png'))
            image.paste(patch_rect, (x,y))
            name = os.path.splitext(img_name)[0]
            image.save(os.path.join(output_folder_path, name + '.png'))
            #data_descripition_file.write(name + "," + str(l) + "," + str(b) + "," + str(x) + ","+str(y)+"\n")
    #data_descripition_file.close()

In [15]:
# make trian directory
os.mkdir('./train')
# get list of all files inside datasett celebA
# that is loaded on kaggle from their database to working area
original = os.listdir("../input/celeba/img_align_celeba")

# put 20000 files aside for processing
c =0
for i in original:
    c+=1
    if c%1000==0:
        print(c)
    if c < 20000:
        file = os.path.join("../input/celeba/img_align_celeba", i)
        shutil.copy(file, './train/')
    else:
       break

In [27]:
os.remove('./test')

In [28]:
# seperate out 15 percent for vaslidation
os.mkdir('./test')

k = 20 # test percent
tst_size = (int(20000)*k/100)
original = os.listdir("./train/")

c=0
for i in original:
    c+=1
    if c%1000==0:
        print(c)
    if c < tst_size:
        file = os.path.join("./train", i)
        shutil.copy(file, './test')
    else:
       break

In [29]:
# apply masking on above train data 
os.mkdir('./mask')
input_folder_path = "./test" # unmasked original photos
output_folder_path = "./mask" # masking photos saved in this folder

prepare_data_set(input_folder_path, output_folder_path, 1)

In [ ]:
a = os.listdir('./mask/')

In [30]:
# randomly pick few images to test and visulaize
# remove them from the test and mask folder both
os.mkdir('./prediction')

original = os.listdir("./mask/")
lt = np.random.choice(original,5,replace=False)
# files were fixed after random selection so same files
# can be used for testing in other models
#lt = ['000125.png','000139.png','000168.png','000178.png','000182.png']

c=0
for i in lt:
  c=c+1
  print(c)
  file = os.path.join("./mask/", i)
  file1 = os.path.join("./test/", i[:-3]+'jpg')
  shutil.copy(file, './prediction')
  os.remove(file) 
  os.remove(file1)

In [31]:
# after doing all this
# train mask and prediction 
# folder were used to create an own dataset on kaggle server
# so that for each execution of the file they are not being created again and again

## Auto-encoder


In [32]:
# Load data using following class
class CustomDataSet(Dataset):
    def __init__(self, dir, transform):
        self.dir = dir
        self.transform = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])
        self.imgs = os.listdir(dir)
        self.imgs.sort()

        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.dir, self.imgs[idx])
        image = Image.open(img_path).convert("RGB")
        img = self.transform(image)
        return img

In [33]:
transform = transforms.Compose( [transforms.Resize((64,64)),transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]),transforms.ToTensor()])

data2 = CustomDataSet('./test',transform=transform)
data3 = CustomDataSet('./mask',transform=transform)

In [34]:
# print i index of data3
p=(data3[6].cpu().detach().numpy())
plt.imshow(np.transpose(p, (1, 2, 0)))

In [35]:
# print i index of data 2 to check both images are same( original and masked ) or not
p=(data2[6].cpu().detach().numpy())
plt.imshow(np.transpose(p, (1, 2, 0)))

In [36]:
# print i index of data3
p=(data3[60].cpu().detach().numpy())
plt.imshow(np.transpose(p, (1, 2, 0)))

In [37]:
# print i index of data 2 to check both images are same( original and masked ) or not
p=(data2[60].cpu().detach().numpy())
plt.imshow(np.transpose(p, (1, 2, 0)))

In [39]:
test_data = []
for i in range(len(data3)):
    if i%1000==0:
        print(i)
    test_data.append([data3[i], data2[i]])

In [40]:
# transform
from torchvision import datasets, transforms
transform = transforms.Compose( [transforms.Resize((64,64)),transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]),transforms.ToTensor()])

dataset = CustomDataSet('./train',transform=transform)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, num_workers=2)
valloader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=2,shuffle=True)

In [41]:
# 4096
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.inp = nn.Linear(12288, 4096) # Input layer
        self.h1 = nn.Linear(4096, 4096) # First Hidden Layer
        self.h5 = nn.Linear(4096, 12288)   # Fifth Hidden layer

    # ouput the result received after applying the actiavation function
    def forward(self, x):
        inpu = F.relu(self.inp(x)) # activation function of first input layer
        l1 = F.relu(self.h1(inpu))
        l5 = F.relu(self.h5(l1))
        return l5
        
net = Net().to(DEVICE)

In [ ]:
# # 2048
# class Net(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.inp = nn.Linear(12288, 4096) # Input layer
#         self.h1 = nn.Linear(4096, 2048) # First Hidden Layer

#         self.h4 = nn.Linear(2048, 4096) # Fourth Hidden layer
#         self.h5 = nn.Linear(4096, 12288)   # Fifth Hidden layer

#     # ouput the result received after applying the actiavation function
#     def forward(self, x):
#         inpu = F.relu(self.inp(x)) # activation function of first input layer
#         l1 = F.relu(self.h1(inpu))

#         l4 = F.relu(self.h4(l1))
#         l5 = F.relu(self.h5(l4))
#         return l5
        
# net = Net().to(DEVICE)

In [ ]:
# # 1024
# class Net(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.inp = nn.Linear(12288, 4096) # Input layer
#         self.h1 = nn.Linear(4096, 2048) # First Hidden Layer
#         self.h2 = nn.Linear(2048, 1024) # Second Hidden layer
#         self.h3 = nn.Linear(1024, 2048) # Third hidden layer
#         self.h4 = nn.Linear(2048, 4096) # Fourth Hidden layer
#         self.h5 = nn.Linear(4096, 12288)   # Fifth Hidden layer

#     # ouput the result received after applying the actiavation function
#     def forward(self, x):
#         inpu = F.relu(self.inp(x)) # activation function of first input layer
#         l1 = F.relu(self.h1(inpu))
#         l2 = F.relu(self.h2(l1))
#         l3 = F.relu(self.h3(l2))
#         l4 = F.relu(self.h4(l1))
#         l5 = F.relu(self.h5(l4))
#         return l5
        
# net = Net().to(DEVICE)

In [ ]:
# # 512
# class Net(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.inp = nn.Linear(12288, 4096) # Input layer
#         self.h1 = nn.Linear(4096, 2048) # First Hidden Layer
#         self.h2 = nn.Linear(2048, 1024) # Second Hidden layer
#         self.h21 = nn.Linear(1024, 512) # Second Hidden layer
#         self.h22 = nn.Linear(512, 1024) # Second Hidden layer   
#         self.h3 = nn.Linear(1024, 2048) # Third hidden layer
#         self.h4 = nn.Linear(2048, 4096) # Fourth Hidden layer
#         self.h5 = nn.Linear(4096, 12288)   # Fifth Hidden layer

#     # ouput the result received after applying the actiavation function
#     def forward(self, x):
#         inpu = F.relu(self.inp(x)) # activation function of first input layer
#         l1 = F.relu(self.h1(inpu))
#         l2 = F.relu(self.h2(l1))
#         l21 = F.relu(self.h21(l2))
#         l22 = F.relu(self.h22(l21))
   
#         l3 = F.relu(self.h3(l22))
#         l4 = F.relu(self.h4(l1))
#         l5 = F.relu(self.h5(l4))
#         return l5
        
# net = Net().to(DEVICE)

In [ ]:
# # 256
# class Net(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.inp = nn.Linear(12288, 4096) # Input layer
#         self.h1 = nn.Linear(4096, 2048) # First Hidden Layer
#         self.h2 = nn.Linear(2048, 1024) # Second Hidden layer
#         self.h21 = nn.Linear(1024, 512) # Second Hidden layer
#         self.h211 = nn.Linear(512, 256) # Second Hidden layer
#         self.h212 = nn.Linear(256, 512) # Second Hidden layer   
#         self.h22 = nn.Linear(512, 1024) # Second Hidden layer   
#         self.h3 = nn.Linear(1024, 2048) # Third hidden layer
#         self.h4 = nn.Linear(2048, 4096) # Fourth Hidden layer
#         self.h5 = nn.Linear(4096, 12288)   # Fifth Hidden layer

#     # ouput the result received after applying the actiavation function
#     def forward(self, x):
#         inpu = F.relu(self.inp(x)) # activation function of first input layer
#         l1 = F.relu(self.h1(inpu))
#         l2 = F.relu(self.h2(l1))
#         l21 = F.relu(self.h21(l2))
#         l211 = F.relu(self.h211(l21))
#         l212 = F.relu(self.h212(l211))
        
#         l22 = F.relu(self.h22(l212))
   
#         l3 = F.relu(self.h3(l22))
#         l4 = F.relu(self.h4(l1))
#         l5 = F.relu(self.h5(l4))
#         return l5
        
# net = Net().to(DEVICE)

In [42]:
loss_fn = nn.MSELoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)
samay = time()
epochs = 10

tr_los = []
los = []
for e in range(epochs):
    epoch_loss = 0
    c=0
    for images in trainloader:
        #print(c)
        c=c+1
        images = images.view(images.shape[0], -1)   
        images = images.to(DEVICE)
        ##  Forward pass
        # get output for the images
        output = net(images)
        
        ## Calculate Loss
        loss = loss_fn(output, images)

        ## Backward pass
        # Training pass- gradient zero initially
        optimizer.zero_grad()

        ## Backpropagation step 
        loss.backward()
        
        ## And optimizes its weights here
        optimizer.step()
        
        # loss.item() 
        epoch_loss += loss.item()
        
    else:     # it is executed when all the samles from training data has been processed and nothinig left at the end
        print("Epoch ",e," - Training loss: ", epoch_loss/len(trainloader))
        tr_los.append(epoch_loss/len(trainloader))
        #---- TEST ----

    sahi = 0
    total = 0
    ll = []
    with torch.no_grad():
        print('test')
        for i,j in valloader:
            images1 = i.view(i.shape[0], -1)   
            images1 = images1.to(DEVICE)
            output = net(images1)
            
            imag = j.view(j.shape[0], -1)   
            imag = imag.to(DEVICE)

            #print(output)

            # check for each sample in a batch although here my test batch is 1 but I tried various batch sizzes for test
            # set all gave same result aas there is no effect of batch size during testing
            loss = loss_fn(output, imag)
            ll.append(loss.cpu().detach().numpy())
            #print(loss)
    # training time printed to check how many minutes it take to run each epoch  
    ll = np.array(ll)
    los.append(ll.mean())
    print(ll.mean())
    print("\nTraining Time (minutes) =",(time()-samay)/60)

In [47]:
## Predcition
# Prediction folder include all the images to be predictive 

dataset2 = CustomDataSet('./prediction',transform=transform)
ader = torch.utils.data.DataLoader(dataset2, num_workers=2)

In [48]:
# predict images and save them

c=0
for i in ader:
    c+=1
    imag_n = i[0].numpy()
   # plt.imshow(np.transpose(imag_n, (1, 2, 0)))
    with torch.no_grad():
        images1 = i.view(i.shape[0], -1)   
        images1 = images1.to(DEVICE)
        output = net(images1)
        p = output.cpu().detach().numpy()
        
    a = output.view((3,64,64),-1)
    p=(a.cpu().detach().numpy())
    plt.figure(figsize=(10,10))
    plt.subplot(len(ader),1,c)
    plt.imshow(np.transpose(p, (1, 2, 0)))
    name = str(c)+'.jpg'
    plt.savefig(name)

